# Comparative metagenomics using SIMKA

This notebook will walk you through how to run a de novo comparative metagenomics analysis using SIMKA

Step 1: Run SIMKA on the read files for your set & create visualizations


## Getting Started

You will need to rerun this section each time you come back to this notebook to reset all directories and variables.

In [ ]:
# set the variables for your netid
netid = "YOUR_NETID"
setid = "YOUR_SETID"

In [ ]:
# Go into the working directory
work_dir = "/xdisk/bhurwitz/bh_class/" + netid + "/assignments/20_kmer_comparisons"
%cd $work_dir

## Creating a config file
Let's create a config file with all of the variables we will need in the scripts below. Then when we want to use these variables in the script, we will "source" the config file to set the variables.

In [ ]:
# create a config file with all of the variables you need
!echo "export NETID=$netid" > config.sh
!echo "export SETID=$setid" >> config.sh
!echo "export WORK_DIR=/xdisk/bhurwitz/bh_class/$netid/assignments/20_kmer_comparisons" >> config.sh
!echo "export SIMKA=/contrib/singularity/shared/bhurwitz/simka:1.5.3--hdcf5f25_4.sif" >> config.sh
!echo "export KMER=31" >> config.sh

In [ ]:
# check the config file to be sure it is correct
# Is your netid and xfile correct? Do you have the right directories?
!cat config.sh

## Step 1: Running SIMKA on the fastq files for all of the samples in your set

In this step, we will run SIMKA to do an all-vs-all sequence comparsion of all of the read files in your set.


In [ ]:
# Create a run script to run simka on all fastq files
my_code = '''#!/bin/bash
#SBATCH --nodes=1
#SBATCH --ntasks=4        
#SBATCH --partition=standard
#SBATCH --account=bhurwitz                       
#SBATCH --output=Job-simka.out
#SBATCH --mem=24gb
#SBATCH --time=48:00:00 

pwd; hostname; date

module load python/3.9/3.9.10
module load R/4.2.2

# load environment
CONDA="/groups/bhurwitz/miniconda3"
source $CONDA/etc/profile.d/conda.sh
conda activate simka

source $SLURM_SUBMIT_DIR/config.sh

cd ${WORK_DIR}
FILE_LIST=${SETID}_simka_list
TEMP=${SETID}_simka_temp
META=${SETID}_simka_meta

mkdir simka_${KMER}_results

### run simka

simka \
-kmer-size ${KMER} \
-in ${FILE_LIST} \
-out simka_${KMER}_results \
-out-tmp ${TEMP} \
-nb-cores 128 \
-count-file ./simka_count.sh \
-merge-file ./simka_merge.sh \
-count-cmd 'sbatch' \
-merge-cmd 'sbatch' \
-max-count 28 \
-max-merge 28 \
-max-memory 768000

### run visualization

python /groups/bhurwitz/tools/simka/visualization/run-visualization.py \
-in simka_${KMER}_results \
-out simka_${KMER}_figures \
-metadata-in ${META} \
-metadata-variable type \
-pca -heatmap -tree

echo "Finished `date`"

'''

with open('run_simka.sh', mode='w') as file:
    file.write(my_code)

In [ ]:
# Create a script to make simka count kmers in parallel on the HPC
my_code = '''#!/bin/bash
#SBATCH --nodes=1
#SBATCH --ntasks=4        
#SBATCH --partition=standard
#SBATCH --account=bhurwitz                       
#SBATCH --output=Job-ct-simka.out
#SBATCH --mem=24gb
#SBATCH --time=12:00:00 

'''

with open('simka_count.sh', mode='w') as file:
    file.write(my_code)

In [ ]:
# Create a script to make simka count kmers in parallel on the HPC
my_code = '''#!/bin/bash
#SBATCH --nodes=1
#SBATCH --ntasks=4        
#SBATCH --partition=standard
#SBATCH --account=bhurwitz                       
#SBATCH --output=Job-merge-simka.out
#SBATCH --mem=24gb
#SBATCH --time=12:00:00 

'''

with open('simka_merge.sh', mode='w') as file:
    file.write(my_code)

In [ ]:
# you should be in your working directory when you run this script
# do you see your config.sh file, and the run_simka.sh, simka_merge.sh, and simka_count.sh scripts?
!pwd
!ls

In [36]:
# Let's run sbatch to run kraken-biom
!sbatch run_simka.sh

Submitted batch job 8598784


In [ ]:
# Welcome back
# You can check if it is running using the squeue command
# Check for all jobs under your netid
!squeue --user=$netid

In [ ]:
# You can check to see if there are any errors by looking at one of the job output files
!cat Job-simka.out

In [ ]:
# copy the visualizations over to your home directory to check them out in jupyter
!cp -r ./simka_31_figures ~

## Final Step
Copy your notebook to the current working directory

In [ ]:
cp ~/hw20_kmer_comparisons.ipynb $work_dir